# Demonstration of Autocorrelation for an Ensemble 
The standard definition of autocorrelation uses the joint probability density function to determine the correlation at a specific delay.  In a practical scenario, we have an ensemble of realisations.  While the relationship at a given delay may not be obvious for all realisations, once averaging of the relationships is carried out, the correlation can be observed.

In this animated demonstration, random signals are generated by driving a bandpass filter with white noise.  For each noise signal, the ouptut will be different.  We will examine the relationship between two points in time, and plot their values on a scatterplot.  Sample points that are correlated will be observed as a linear relationship between the samples.  If, however, they are not correlated, no relationship will be evident.

The original code was produced by B. Mulgrew, 2016

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sps
import time
import pylab as pl
from IPython import display

### User specified parameters
The following parameters can be specified. 

Parameter | Meaning
--------- | -------
<code>N</code>| Length of the generating bandpass filter (e.g. 128)
<code>length</code> | Length of the signals being collected (e.g. 200)
<code>num_ensemble</code> | Number of signals in the ensemble (e.g. 100)
<code>t0</code> | Time of first sample (must be less than <code>length</code>)
<code>t1</code> | Time of second sample (must be less than <code>length</code>)

In [ ]:
# no. of coefficients in FIR filter
N = 128

# Define the number of signal samples to be generated
length = 200

# no. of realisations of ensemble
num_ensemble = 100

# Define the timepoints of interest
t0 = 64
t1 = 73

###  Define a band pass filter (BPF) for generating the sequence
By using an FIR filter, we know the length of the filter startup phase, so know how many points to ignore from the start of the output sequence.

In [ ]:
# BPF spec; sampling frequency normalised to 2
Wn = [0.303, 0.353]

# window design of FIR
B = sps.firwin(N+1, Wn, pass_zero = False)

# normalise white noise gain of filter to 1
B = B / np.sqrt(np.dot(B, B.T))

# Define the likely limit of the filter output 
# if the input is Gaussian with unit variance
plot_limit = 3

Also, define a function that produces filtered noise at the output using the filter we have just defined.

In [ ]:
def filter_output(length, B):
    # Define a Gaussian noise sequence of unit variance
    x = np.random.randn(length + len(B) -1)
    
    # Filter the random sequence
    y = sps.lfilter(B, 1, x)
    
    # Return the filter ouptut after the transient response
    return(y[-length:])

### Theoretical autocorrelation
We know that if the input to the generating filter is white noise, with unit variance, then the output autocorrelation is defined by $$r_{yy}=r_{hh}$$.  Using that property, we can find the theoretical autocorrelation from the system impulse response.

In [ ]:
# from BPF impulse response since 
# input is unit variance white noise

# Use numpy.correlate for autocorrelation of B
R_yy = np.correlate(B, B, 'full')    

# lags for autocorrelation plot
lag = np.arange(-N, N+1)           

# Update figure size and label font size
plt.figure(figsize = (16, 8))
plt.rcParams.update({'font.size': 16})

# Plot the figure
plt.stem(lag, R_yy, use_line_collection=True)

# FIR filter of N coefficients
plt.xlim([-N, N])
# size of y-axis is known because of signal generation mechanism
# unit variance signal
plt.ylim([-1, 1])

plt.xlabel('Lag, l')
plt.ylabel('$\gamma_{yy}(l)$')
plt.title('Autocorrelation');

### Ensemble of signals

In [ ]:
# Define the number of realisations, 
# and create space to store scatterplot results

# array for scatterplot
XX = np.zeros((2, num_ensemble))

# for loop act as animation 
for ii in range(0, num_ensemble):
    # Every time around this loop, a new realisation is obtained
    # and analysed

    # Obtain one realisation of the system output
    y = filter_output(length+N, B)
    
    ### Produce the output figures
    
    pl.figure(figsize=(16, 8))
    pl.rcParams.update({'font.size': 16})
    
    ### Display the signal
    
    # Select the left hand graph
    ax1 = pl.subplot(1, 2, 1)     
    # plot output samples after transient response has decayed away
    ax1.plot(y)                
    
    # We want to annotate the graph, so allow overplotting
    
    # highlight particlar points in time
    ax1.plot(t0, y[t0], 'rx')    
    ax1.plot(t1, y[t1], 'rx')
    # There is a high probability that -3 < y < 3 as y has unit variance
    pl.axis([0, length, -plot_limit, plot_limit])
    pl.xlabel('Sample, n')
    pl.ylabel('x(n)')
    pl.title('One realisation from the ensemble')
    # Show the grid lines
    pl.grid(True)
    
    ### Produce the scatterplot
    
    # add data from this realisation to the end of the list of
    # previous realisations
    XX[0][ii] = y[t0]     
    XX[1][ii] = y[t1]
    
    # Select the right hand graph
    ax2 = pl.subplot(1, 2, 2)    
    
    # Plot all of the previously collected points with blue crosses, 
    # highlight the newest point (ii) with a red circle
    ax2.plot(XX[0][:ii], XX[1][:ii], 'bx',
             XX[0][ii], XX[1][ii], 'ro')
    
    # Plot all of the samples at time t0 along the bottom edge
    ax2.plot(np.squeeze(XX[0][:ii+1]),
             -(plot_limit-0.2)*np.ones(ii+1),
             'g+')
    
    # Connect the last point from the bottom edge to 
    # the scatterplot with a dotted line
    ax2.plot([XX[0][ii], XX[0][ii]],
             [-(plot_limit-0.2), XX[1][ii]],
             'r:')
    
    # And do the same for t1 along the left hand edge
    ax2.plot(-(plot_limit-0.2)*np.ones(ii+1),
             np.squeeze(XX[1][:ii+1]),
             'g+')
    ax2.plot([-(plot_limit-0.2), XX[0][ii]],
             [XX[1][ii], XX[1][ii]],
             'r:')

    # Label the axes
    pl.xticks(np.linspace(-plot_limit, plot_limit, 7))
    pl.axis([-plot_limit, plot_limit, -plot_limit, plot_limit])
    ax2.set_xlabel('x(%d)'%t0)
    ax2.set_ylabel('x(%d)'%t1)
    
    pl.title('Scatterplot')
    pl.grid(True)
    
    # Automatically adjust subplot parameters to give specified padding
    pl.tight_layout()
    display.clear_output(wait=True)
    
    pl.pause(0.2)           # 0.2 second pause between plots
        
print ('Completed %d realisations.'%num_ensemble)      

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.